In [1]:
import os
from PIL import Image
import numpy as np
from torchvision import transforms
import torch

In [ ]:
from pathlib import Path
import os

# Define paths
IMAGE_FOLDER = Path(r"C:\Users\vivek\OneDrive\Desktop\Multimodal Conversational AI\temp llava\images")
output_dir = Path(r"C:\Users\vivek\OneDrive\Desktop\Multimodal Conversational AI\temp llava\normalized_images")
image_files = ["chest_xray_001.jpg", "chest_xray_002.jpg", "chest_xray_003.jpg"]

In [11]:
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Define paths and variables
MODEL_ID = "llava-hf/llava-1.5-7b-hf"
#DATASET_PATH = "./dataset.json"  # Path to your JSON dataset
DATASET_PATH = Path(r"C:\Users\vivek\OneDrive\Desktop\Multimodal Conversational AI\temp llava\openi-instruct.json")
#IMAGE_FOLDER = "./dataset/images"  # Folder containing images
OUTPUT_DIR = "llava-7b-finetuned"
MAX_LENGTH = 256  # Adjust based on your dataset's token length
LOG_FILE = os.path.join(OUTPUT_DIR, "training_log.txt")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoProcessor,
    LlavaForConditionalGeneration,
    TrainingArguments,
    Trainer,
)

In [ ]:
# Load model and processor on CPU
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,  # Full precision for CPU
    device_map="cpu",
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Load and preprocess dataset
dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

def preprocess_function(examples):
    images = []
    input_ids = []
    labels = []

    for example in zip(examples["image"], examples["conversations"]):
        image_path = os.path.join(IMAGE_FOLDER, example[0])
        conversation = example[1]

        # Format conversation into a prompt
        prompt = ""
        for turn in conversation:
            role = "USER" if turn["from"] == "human" else "ASSISTANT"
            prompt += f"{role}: {turn['value']}\n"
            if role == "USER" and "<image>" in turn["value"]:
                prompt = prompt.replace("<image>", "")

        # Process inputs
        inputs = processor(
            text=prompt,
            images=[image_path],
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LENGTH,
            padding="max_length",
        )

        images.append(inputs["pixel_values"].squeeze(0))
        input_ids.append(inputs["input_ids"].squeeze(0))
        labels.append(inputs["input_ids"].squeeze(0))  # Labels are same as input_ids for causal LM

    return {
        "pixel_values": images,
        "input_ids": input_ids,
        "labels": labels,
    }

# Apply preprocessing
dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

# Define training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,  # Small batch size for CPU
    gradient_accumulation_steps=8,  # Accumulate to simulate larger batch size
    learning_rate=1e-5,  # Lower learning rate for stability
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    logging_dir=OUTPUT_DIR,
    logging_strategy="steps",
    log_level="info",
    report_to="none",  # Disable Weights & Biases
    no_cuda=True,  # Explicitly disable GPU
    fp16=False,  # No mixed precision on CPU
    evaluation_strategy="no",  # Add validation set if available
)

# Custom logging to file
def log_to_file(trainer, step, metrics):
    with open(LOG_FILE, "a") as f:
        f.write(f"Step {step}: {metrics}\n")

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda x: {
        "pixel_values": torch.stack([item["pixel_values"] for item in x]),
        "input_ids": torch.stack([item["input_ids"] for item in x]),
        "labels": torch.stack([item["labels"] for item in x]),
    },
)

# Override logging to write to file
trainer.log = lambda logs: log_to_file(trainer, trainer.state.global_step, logs)

# Start fine-tuning
trainer.train()

# Save the model
model.save_pretrained(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 28253708288 bytes.